In [46]:
import datetime
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px # to plot the time series plot
from sklearn import metrics # for the evaluation
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import tensorflow as tf 
from scipy import stats

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
import itertools

import warnings
warnings.filterwarnings("ignore")

# I. Load data

In [47]:
OF = pd.read_pickle("Related_dataset/Opinion_Finder_result.pkl")
GPOMS = pd.read_pickle("Related_dataset/GPOMS_result.pkl")
DJIA = pd.read_pickle("Related_dataset/DJIA_filled_missing_value.pkl")

# 2. Prepare dataset

In [48]:
# process GPOMS dataframe
GPOMS_pivot = GPOMS.drop(columns=['tweets_processed', 'tweet_size', 'GPOMS_preprocess_data', 'GPOMS_sum','sum_index', 'avg_index'])
GPOMS_pivot[['Calm','Kind','Happy','Alert','Sure','Vital']] = pd.DataFrame(GPOMS.GPOMS_avg.tolist(), index= GPOMS.index)
GPOMS_pivot = GPOMS_pivot.drop(['GPOMS_avg'], axis=1)
GPOMS_pivot = GPOMS_pivot.reset_index()
GPOMS_pivot = GPOMS_pivot.rename({'date_prune': 'Dates'}, axis=1) 

# process DJIA dataframe
DJIA['Dates'] = DJIA['Dates'].astype(str)

combine_df = DJIA.merge(GPOMS_pivot, on='Dates')
combine_df

,Dates,Close,Calm,Kind,Happy,Alert,Sure,Vital
0,2020-04-09,23719.369141,0.073364,0.191523,-0.115305,0.346767,0.136158,0.155670
1,2020-04-10,23555.069336,0.092862,0.195467,-0.102743,0.342314,0.130042,0.118410
2,2020-04-11,23472.919434,0.061887,0.278887,-0.064833,0.340592,0.101464,0.160457
3,2020-04-12,23431.844482,0.082763,0.216780,-0.062001,0.329557,0.111304,0.079260
4,2020-04-13,23390.769531,0.079693,0.222743,-0.091400,0.338160,0.130949,0.137338
...,...,...,...,...,...,...,...,...
72,2020-07-12,26083.175781,0.078391,0.181213,-0.097056,0.369545,0.147573,0.078480
73,2020-07-13,26085.800781,0.082311,0.183053,-0.082040,0.357759,0.154204,0.122646
74,2020-07-14,26642.589844,0.098220,0.186504,-0.089044,0.359141,0.150973,0.111742
75,2020-07-15,26870.099609,0.084568,0.184610,-0.079527,0.337375,0.146182,0.102206


# III. Build Neural Network model to predict DJIA
input: 
1. The past 5 days of DJIA values
2. Opinion Finder mood data (past 5 days)

Time period (in thesis): 
> Train: 2008-2-28 ~ 2008-11-28 (10 months)

> Test: 2008-12-1 ~ 2008-12-19 (19 days)

Time period (in this project): 
> Train: 2020-4-9 ~ 2020-6-23 (54 days, some missing data)

> Test: 2020-6-24 ~ 2020-7-16 (18 days)


In [49]:
threshold = 54
WINDOW_SIZE = 5

### Part 2: 提供過去3天的DJIA + GPOMS 資料: 

multi-variable

> X[0]: [[ [1-1, 1-2, ..], [2-1, 2-2, ...], [3-1, 3-2, ...]]]

> y[0]: [4]


In [50]:
temp2 = combine_df.set_index('Dates')
mms_close = MinMaxScaler()
mms_calm = MinMaxScaler()
mms_kind = MinMaxScaler()
mms_happy = MinMaxScaler()
mms_alert = MinMaxScaler()
mms_sure = MinMaxScaler()
mms_vital = MinMaxScaler()
mms_delta = MinMaxScaler()

temp2['Close'] = mms_close.fit_transform(np.array(temp2['Close']).reshape(-1, 1))
temp2['Calm'] = mms_calm.fit_transform(np.array(temp2['Calm']).reshape(-1, 1))
temp2['Kind'] = mms_kind.fit_transform(np.array(temp2['Kind']).reshape(-1, 1))
temp2['Happy'] = mms_happy.fit_transform(np.array(temp2['Happy']).reshape(-1, 1))
temp2['Alert'] = mms_alert.fit_transform(np.array(temp2['Alert']).reshape(-1, 1))
temp2['Sure'] = mms_sure.fit_transform(np.array(temp2['Sure']).reshape(-1, 1))
temp2['Vital'] = mms_vital.fit_transform(np.array(temp2['Vital']).reshape(-1, 1))

temp2.head()

,Close,Calm,Kind,Happy,Alert,Sure,Vital
Dates,,,,,,,
2020-04-09,0.153833,0.710392,0.416849,0.471563,0.458773,0.248355,0.778378
2020-04-10,0.117752,0.838021,0.443179,0.596098,0.437843,0.204577,0.468202
2020-04-11,0.099711,0.635268,1.000000,0.971922,0.429748,0.000000,0.818231
2020-04-12,0.090690,0.771914,0.585441,1.000000,0.377869,0.070439,0.142280
2020-04-13,0.081670,0.751823,0.625244,0.708548,0.418315,0.211071,0.625771


In [51]:
def df_to_X_y2(df, window_size=3): # transfrom input data
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np)-window_size):
        row = [r for r in df_as_np[i:i+window_size]]
        X.append(row)
        label = df_as_np[i+window_size][0]
        y.append(label)
    return np.array(X), np.array(y)

In [52]:
def plot_modal_res(df, tp): # plot the train/test data result
    plt.title(f"{tp} result")
    plt.plot(df[f'{tp} Predictions'], c='b')
    plt.plot(df['Actuals'], c='g')
    plt.legend(["Modal Predict", "Ground True"], loc ="lower right")
    plt.show()

In [53]:
# Calculate the Mape and Directional correction rate
def mape(actual, predict): 
    actual, predict = np.array(actual), np.array(predict)
    return np.mean(np.abs((actual - predict) / actual)) * 100

def direction(actual, predict): 
    prev_act = actual[0]
    prev_pred = predict[0]
    true = []
    res = []
    for i, j in zip(actual[1:], predict[1:]): 
        if i>=prev_act:
            true.append(1)
        else:
            true.append(-1)
        if j>=prev_pred:
            res.append(1)
        else:
            res.append(-1)
        prev_act = i
        prev_pred = j
    denorminator = len(actual) -1
    numerator = 0
    for i, j in zip(true, res):
        if i==j:
            numerator += 1
    return (numerator / denorminator)

***Start iteration***

In [54]:
type = combine_df.columns[2:]
type

Index(['Calm', 'Kind', 'Happy', 'Alert', 'Sure', 'Vital'], dtype='object')

In [55]:
def eval(results):
    MAPE = mape(results['Actuals'], results['Test Predictions']).round(3)
    Direction =  direction(results['Actuals'], results['Test Predictions'])
    return MAPE, Direction

In [56]:
finish = []

num = [2, 3, 4, 5, 6, 7]
for j in num:
    NUM_FEATURE = j
    sum = list(itertools.combinations(type, NUM_FEATURE-1))

    for i in sum:
        k = ['Close']+list(i)
        tmp = temp2[k]
        
        X2, y2 = df_to_X_y2(tmp, WINDOW_SIZE)
        X2.shape, y2.shape # number of data, # size of window, number of features

        X_train2, y_train2 = X2[:threshold-WINDOW_SIZE], y2[:threshold-WINDOW_SIZE]
        X_test2, y_test2 = X2[threshold:], y2[threshold:]

        model2 = Sequential()
        model2.add(InputLayer((WINDOW_SIZE, NUM_FEATURE)))
        model2.add(LSTM(64))
        model2.add(Dense(8, 'relu'))
        model2.add(Dense(1, 'linear'))
        model2.summary()

        model2.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.01), metrics=[RootMeanSquaredError()])
        model2.fit(X_train2, y_train2, epochs=10)

        # train result
        train_predictions2 = model2.predict(X_train2).flatten()
        pred = mms_close.inverse_transform(train_predictions2.reshape(-1, 1))
        pred = pred.reshape(-1,)
        act = mms_close.inverse_transform(y_train2.reshape(-1, 1))
        act = act.reshape(-1,)

        train_results2 = pd.DataFrame(data={
            'Train Predictions':pred, 
            'Actuals':act})

        # Test result
        test_predictions2 = model2.predict(X_test2).flatten()
        pred = mms_close.inverse_transform(test_predictions2.reshape(-1, 1))
        pred = pred.reshape(-1,)
        act = mms_close.inverse_transform(y_test2.reshape(-1, 1))
        act = act.reshape(-1,)

        test_results2 = pd.DataFrame(data={
            'Test Predictions':pred, 
            'Actuals':act})
        
        e1, e2 = eval(test_results2)
        finish.append([i, e1, e2])
        del model2

Model: "sequential_763"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_763 (LSTM)             (None, 64)                17152     
                                                                 
 dense_1526 (Dense)          (None, 8)                 520       
                                                                 
 dense_1527 (Dense)          (None, 1)                 9         
                                                                 
Total params: 17,681
Trainable params: 17,681
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
2/2 [==============================] - 4s 5ms/step - loss: 0.1282 - root_mean_squared_error: 0.3580
Epoch 2/10
2/2 [==============================] - 0s 5ms/step - loss: 0.1214 - root_mean_squared_error: 0.3484
Epoch 3/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0436 - root_mean_s

 Layer (type)                Output Shape              Param #   
 lstm_767 (LSTM)             (None, 64)                17152     
                                                                 
 dense_1534 (Dense)          (None, 8)                 520       
                                                                 
 dense_1535 (Dense)          (None, 1)                 9         
                                                                 
Total params: 17,681
Trainable params: 17,681
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
2/2 [==============================] - 3s 5ms/step - loss: 0.2888 - root_mean_squared_error: 0.5374
Epoch 2/10
2/2 [==============================] - 0s 6ms/step - loss: 0.1612 - root_mean_squared_error: 0.4015
Epoch 3/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0538 - root_mean_squared_error: 0.2319
Epoch 4/10
2/2 [==============================] - 0s 5ms/step - loss:

 lstm_771 (LSTM)             (None, 64)                17408     
                                                                 
 dense_1542 (Dense)          (None, 8)                 520       
                                                                 
 dense_1543 (Dense)          (None, 1)                 9         
                                                                 
Total params: 17,937
Trainable params: 17,937
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
2/2 [==============================] - 2s 6ms/step - loss: 0.3697 - root_mean_squared_error: 0.6080
Epoch 2/10
2/2 [==============================] - 0s 7ms/step - loss: 0.2640 - root_mean_squared_error: 0.5138
Epoch 3/10
2/2 [==============================] - 0s 6ms/step - loss: 0.2457 - root_mean_squared_error: 0.4957
Epoch 4/10
2/2 [==============================] - 0s 5ms/step - loss: 0.2303 - root_mean_squared_error: 0.4798
Epoch 5/10
2/2 [========

 lstm_775 (LSTM)             (None, 64)                17408     
                                                                 
 dense_1550 (Dense)          (None, 8)                 520       
                                                                 
 dense_1551 (Dense)          (None, 1)                 9         
                                                                 
Total params: 17,937
Trainable params: 17,937
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
2/2 [==============================] - 3s 6ms/step - loss: 0.2163 - root_mean_squared_error: 0.4651
Epoch 2/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0608 - root_mean_squared_error: 0.2467
Epoch 3/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0517 - root_mean_squared_error: 0.2273
Epoch 4/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0536 - root_mean_squared_error: 0.2316
Epoch 5/10
2/2 [========

                                                                 
 dense_1558 (Dense)          (None, 8)                 520       
                                                                 
 dense_1559 (Dense)          (None, 1)                 9         
                                                                 
Total params: 17,937
Trainable params: 17,937
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
2/2 [==============================] - 3s 5ms/step - loss: 0.3560 - root_mean_squared_error: 0.5967
Epoch 2/10
2/2 [==============================] - 0s 5ms/step - loss: 0.1138 - root_mean_squared_error: 0.3373
Epoch 3/10
2/2 [==============================] - 0s 5ms/step - loss: 0.1266 - root_mean_squared_error: 0.3558
Epoch 4/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0451 - root_mean_squared_error: 0.2123
Epoch 5/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0591 - root_mean_s

...

_________________________________________________________________
Epoch 1/10
2/2 [==============================] - 2s 6ms/step - loss: 0.1563 - root_mean_squared_error: 0.3953
Epoch 2/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0661 - root_mean_squared_error: 0.2570
Epoch 3/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0515 - root_mean_squared_error: 0.2269
Epoch 4/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0335 - root_mean_squared_error: 0.1831
Epoch 5/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0379 - root_mean_squared_error: 0.1946
Epoch 6/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0186 - root_mean_squared_error: 0.1362
Epoch 7/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0351 - root_mean_squared_error: 0.1873
Epoch 8/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0227 - root_mean_squared_error: 0.1505
Epoch 9/10
2/2 [==============================

Epoch 1/10
2/2 [==============================] - 3s 5ms/step - loss: 0.3631 - root_mean_squared_error: 0.6026
Epoch 2/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0911 - root_mean_squared_error: 0.3018
Epoch 3/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0813 - root_mean_squared_error: 0.2852
Epoch 4/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0637 - root_mean_squared_error: 0.2523
Epoch 5/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0661 - root_mean_squared_error: 0.2571
Epoch 6/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0419 - root_mean_squared_error: 0.2047
Epoch 7/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0482 - root_mean_squared_error: 0.2195
Epoch 8/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0372 - root_mean_squared_error: 0.1928
Epoch 9/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0290 - root_mean_squared_error: 0.1702
E

2/2 [==============================] - 2s 5ms/step - loss: 0.1574 - root_mean_squared_error: 0.3968
Epoch 2/10
2/2 [==============================] - 0s 5ms/step - loss: 0.1631 - root_mean_squared_error: 0.4039
Epoch 3/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0562 - root_mean_squared_error: 0.2370
Epoch 4/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0742 - root_mean_squared_error: 0.2723
Epoch 5/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0529 - root_mean_squared_error: 0.2299
Epoch 6/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0385 - root_mean_squared_error: 0.1963
Epoch 7/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0405 - root_mean_squared_error: 0.2012
Epoch 8/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0252 - root_mean_squared_error: 0.1586
Epoch 9/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0241 - root_mean_squared_error: 0.1554
Epoch 10/10


2/2 [==============================] - 2s 5ms/step - loss: 0.1946 - root_mean_squared_error: 0.4411
Epoch 2/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0791 - root_mean_squared_error: 0.2812
Epoch 3/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0670 - root_mean_squared_error: 0.2588
Epoch 4/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0514 - root_mean_squared_error: 0.2266
Epoch 5/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0337 - root_mean_squared_error: 0.1835
Epoch 6/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0332 - root_mean_squared_error: 0.1823
Epoch 7/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0208 - root_mean_squared_error: 0.1441
Epoch 8/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0231 - root_mean_squared_error: 0.1519
Epoch 9/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0182 - root_mean_squared_error: 0.1349
Epoch 10/10


2/2 [==============================] - 2s 6ms/step - loss: 0.1559 - root_mean_squared_error: 0.3948
Epoch 2/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0785 - root_mean_squared_error: 0.2802
Epoch 3/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0613 - root_mean_squared_error: 0.2476
Epoch 4/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0439 - root_mean_squared_error: 0.2096
Epoch 5/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0353 - root_mean_squared_error: 0.1879
Epoch 6/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0249 - root_mean_squared_error: 0.1576
Epoch 7/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0208 - root_mean_squared_error: 0.1441
Epoch 8/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0194 - root_mean_squared_error: 0.1393
Epoch 9/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0258 - root_mean_squared_error: 0.1606
Epoch 10/10


In [58]:
df_res = pd.DataFrame(finish, columns = ['combination', 'MAPE','Direction'])
df_res

,combination,MAPE,Direction
0,"(Calm,)",2.074,0.470588
1,"(Kind,)",0.936,0.529412
2,"(Happy,)",1.636,0.529412
3,"(Alert,)",0.741,0.705882
4,"(Sure,)",1.100,0.588235
...,...,...,...
58,"(Calm, Kind, Happy, Sure, Vital)",2.902,0.529412
59,"(Calm, Kind, Alert, Sure, Vital)",1.180,0.647059
60,"(Calm, Happy, Alert, Sure, Vital)",0.829,0.647059
61,"(Kind, Happy, Alert, Sure, Vital)",0.911,0.647059


In [60]:
df_res.query(' Direction > 0.6').reset_index(drop=True)

,combination,MAPE,Direction
0,"(Alert,)",0.741,0.705882
1,"(Vital,)",2.301,0.647059
2,"(Calm, Alert)",8.509,0.764706
3,"(Kind, Vital)",1.403,0.647059
4,"(Alert, Sure)",2.835,0.705882
5,"(Alert, Vital)",0.755,0.647059
6,"(Calm, Happy, Alert)",1.232,0.647059
7,"(Alert, Sure, Vital)",2.290,0.647059
8,"(Happy, Alert, Sure, Vital)",1.453,0.647059
9,"(Calm, Kind, Happy, Alert, Vital)",0.958,0.647059


['Kind','Happy','Sure']: lag days不太可能

# Part 2: confidence  interval

In [67]:
candidate = {
    1:[['Alert'], ['Sure']] ,
    2:[['Alert','Sure'], ['Alert','Vital']],
    3:[['Alert','Sure','Vital']],
    4:[['Happy','Alert','Sure','Vital']]
}

best_lagdays = [5,1,4,4,4,3]
it = 0
for features in candidate:
    print('\n', features)
    for comb in candidate[features]:
        print(comb)
        print('->',best_lagdays[it])
        it += 1


 1
['Alert']
-> 5
['Sure']
-> 1

 2
['Alert', 'Sure']
-> 4
['Alert', 'Vital']
-> 4

 3
['Alert', 'Sure', 'Vital']
-> 4

 4
['Happy', 'Alert', 'Sure', 'Vital']
-> 3


In [68]:
feature_num = []
combination = []
MAPE = []
correction = []
best_lagdays = [5,1,4,4,4,3]
it = 0

for features in candidate:
    for comb in candidate[features]:
        WINDOW_SIZE = best_lagdays[it]
        it += 1
        for i in range(100):
            # Data
            NUM_FEATURE = features+1 # close price
            k = ['Close']+comb
            tmp = temp2[k]
            X2, y2 = df_to_X_y2(tmp, WINDOW_SIZE)
            X_train2, y_train2 = X2[:threshold-3], y2[:threshold-3]
            X_test2, y_test2 = X2[threshold:], y2[threshold:]
            
            # Modal
            model_k = Sequential()
            model_k.add(InputLayer((WINDOW_SIZE, NUM_FEATURE)))
            model_k.add(LSTM(64))
            model_k.add(Dense(8, 'relu'))
            model_k.add(Dense(1, 'linear'))
            model_k.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.01), metrics=[RootMeanSquaredError()])
            model_k.fit(X_train2, y_train2, epochs=10)
            
            # Test result
            test_predictions2 = model_k.predict(X_test2).flatten()
            pred = mms_close.inverse_transform(test_predictions2.reshape(-1, 1))
            pred = pred.reshape(-1,)
            act = mms_close.inverse_transform(y_test2.reshape(-1, 1))
            act = act.reshape(-1,)
            test_results2 = pd.DataFrame(data={
                'Test Predictions':pred, 
                'Actuals':act})

            # Part 2: past three days' DJIA data + Opinion Finder data
            feature_num.append(features)
            combination.append(comb)
            MAPE.append(mape(test_results2['Actuals'], test_results2['Test Predictions']).round(3))
            correction.append(direction(test_results2['Actuals'], test_results2['Test Predictions']))
            del model_k

Epoch 1/10
2/2 [==============================] - 2s 5ms/step - loss: 0.1669 - root_mean_squared_error: 0.4086
Epoch 2/10
2/2 [==============================] - 0s 7ms/step - loss: 0.0450 - root_mean_squared_error: 0.2122
Epoch 3/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0266 - root_mean_squared_error: 0.1631
Epoch 4/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0225 - root_mean_squared_error: 0.1501
Epoch 5/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0363 - root_mean_squared_error: 0.1905
Epoch 6/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0229 - root_mean_squared_error: 0.1514
Epoch 7/10
2/2 [==============================] - 0s 7ms/step - loss: 0.0268 - root_mean_squared_error: 0.1637
Epoch 8/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0219 - root_mean_squared_error: 0.1481
Epoch 9/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0204 - root_mean_squared_error: 0.1427
E

2/2 [==============================] - 3s 6ms/step - loss: 0.1360 - root_mean_squared_error: 0.3688
Epoch 2/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0804 - root_mean_squared_error: 0.2835
Epoch 3/10
2/2 [==============================] - 0s 7ms/step - loss: 0.0361 - root_mean_squared_error: 0.1899
Epoch 4/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0576 - root_mean_squared_error: 0.2401
Epoch 5/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0491 - root_mean_squared_error: 0.2217
Epoch 6/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0248 - root_mean_squared_error: 0.1573
Epoch 7/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0313 - root_mean_squared_error: 0.1768
Epoch 8/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0302 - root_mean_squared_error: 0.1736
Epoch 9/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0194 - root_mean_squared_error: 0.1395
Epoch 10/10


2/2 [==============================] - 2s 7ms/step - loss: 0.1521 - root_mean_squared_error: 0.3899
Epoch 2/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0704 - root_mean_squared_error: 0.2654
Epoch 3/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0289 - root_mean_squared_error: 0.1699
Epoch 4/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0461 - root_mean_squared_error: 0.2146
Epoch 5/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0416 - root_mean_squared_error: 0.2040
Epoch 6/10
2/2 [==============================] - 0s 7ms/step - loss: 0.0195 - root_mean_squared_error: 0.1398
Epoch 7/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0317 - root_mean_squared_error: 0.1781
Epoch 8/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0266 - root_mean_squared_error: 0.1631
Epoch 9/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0178 - root_mean_squared_error: 0.1334
Epoch 10/10


2/2 [==============================] - 2s 5ms/step - loss: 0.2819 - root_mean_squared_error: 0.5309
Epoch 2/10
2/2 [==============================] - 0s 6ms/step - loss: 0.1374 - root_mean_squared_error: 0.3706
Epoch 3/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0519 - root_mean_squared_error: 0.2279
Epoch 4/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0756 - root_mean_squared_error: 0.2749
Epoch 5/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0386 - root_mean_squared_error: 0.1965
Epoch 6/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0543 - root_mean_squared_error: 0.2331
Epoch 7/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0535 - root_mean_squared_error: 0.2313
Epoch 8/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0349 - root_mean_squared_error: 0.1869
Epoch 9/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0211 - root_mean_squared_error: 0.1453
Epoch 10/10


2/2 [==============================] - 2s 7ms/step - loss: 0.2621 - root_mean_squared_error: 0.5120
Epoch 2/10
2/2 [==============================] - 0s 7ms/step - loss: 0.0659 - root_mean_squared_error: 0.2566
Epoch 3/10
2/2 [==============================] - 0s 7ms/step - loss: 0.0731 - root_mean_squared_error: 0.2703
Epoch 4/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0336 - root_mean_squared_error: 0.1832
Epoch 5/10
2/2 [==============================] - 0s 7ms/step - loss: 0.0319 - root_mean_squared_error: 0.1785
Epoch 6/10
2/2 [==============================] - 0s 7ms/step - loss: 0.0250 - root_mean_squared_error: 0.1581
Epoch 7/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0244 - root_mean_squared_error: 0.1564
Epoch 8/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0196 - root_mean_squared_error: 0.1400
Epoch 9/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0222 - root_mean_squared_error: 0.1489
Epoch 10/10


...

2/2 [==============================] - 3s 5ms/step - loss: 0.1836 - root_mean_squared_error: 0.4285
Epoch 2/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0748 - root_mean_squared_error: 0.2734
Epoch 3/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0602 - root_mean_squared_error: 0.2454
Epoch 4/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0429 - root_mean_squared_error: 0.2071
Epoch 5/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0306 - root_mean_squared_error: 0.1750
Epoch 6/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0244 - root_mean_squared_error: 0.1562
Epoch 7/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0179 - root_mean_squared_error: 0.1339
Epoch 8/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0226 - root_mean_squared_error: 0.1502
Epoch 9/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0160 - root_mean_squared_error: 0.1264
Epoch 10/10


2/2 [==============================] - 3s 5ms/step - loss: 0.2011 - root_mean_squared_error: 0.4485
Epoch 2/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0665 - root_mean_squared_error: 0.2580
Epoch 3/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0696 - root_mean_squared_error: 0.2638
Epoch 4/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0459 - root_mean_squared_error: 0.2143
Epoch 5/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0521 - root_mean_squared_error: 0.2283
Epoch 6/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0355 - root_mean_squared_error: 0.1885
Epoch 7/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0248 - root_mean_squared_error: 0.1576
Epoch 8/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0263 - root_mean_squared_error: 0.1623
Epoch 9/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0158 - root_mean_squared_error: 0.1257
Epoch 10/10


2/2 [==============================] - 3s 5ms/step - loss: 0.2775 - root_mean_squared_error: 0.5268
Epoch 2/10
2/2 [==============================] - 0s 5ms/step - loss: 0.1880 - root_mean_squared_error: 0.4336
Epoch 3/10
2/2 [==============================] - 0s 4ms/step - loss: 0.1146 - root_mean_squared_error: 0.3386
Epoch 4/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0549 - root_mean_squared_error: 0.2343
Epoch 5/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0920 - root_mean_squared_error: 0.3034
Epoch 6/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0763 - root_mean_squared_error: 0.2762
Epoch 7/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0579 - root_mean_squared_error: 0.2406
Epoch 8/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0585 - root_mean_squared_error: 0.2419
Epoch 9/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0629 - root_mean_squared_error: 0.2508
Epoch 10/10


2/2 [==============================] - 2s 4ms/step - loss: 0.2451 - root_mean_squared_error: 0.4951
Epoch 2/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0851 - root_mean_squared_error: 0.2917
Epoch 3/10
2/2 [==============================] - 0s 5ms/step - loss: 0.1036 - root_mean_squared_error: 0.3219
Epoch 4/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0607 - root_mean_squared_error: 0.2463
Epoch 5/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0543 - root_mean_squared_error: 0.2330
Epoch 6/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0546 - root_mean_squared_error: 0.2336
Epoch 7/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0408 - root_mean_squared_error: 0.2019
Epoch 8/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0289 - root_mean_squared_error: 0.1700
Epoch 9/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0334 - root_mean_squared_error: 0.1829
Epoch 10/10


2/2 [==============================] - 2s 4ms/step - loss: 0.2173 - root_mean_squared_error: 0.4662
Epoch 2/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0712 - root_mean_squared_error: 0.2668
Epoch 3/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0949 - root_mean_squared_error: 0.3081
Epoch 4/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0514 - root_mean_squared_error: 0.2266
Epoch 5/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0508 - root_mean_squared_error: 0.2254
Epoch 6/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0520 - root_mean_squared_error: 0.2280
Epoch 7/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0276 - root_mean_squared_error: 0.1663
Epoch 8/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0296 - root_mean_squared_error: 0.1722
Epoch 9/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0294 - root_mean_squared_error: 0.1714
Epoch 10/10


In [69]:
df_res = pd.DataFrame(
    {'feature_num':feature_num,
     'combination':combination,
     'MAPE': MAPE,
     'Direction': correction})

In [70]:
df_res

,feature_num,combination,MAPE,Direction
0,1,[Alert],2.347,0.647059
1,1,[Alert],0.820,0.470588
2,1,[Alert],2.095,0.647059
3,1,[Alert],1.961,0.647059
4,1,[Alert],1.099,0.647059
...,...,...,...,...
595,4,"[Happy, Alert, Sure, Vital]",2.596,0.473684
596,4,"[Happy, Alert, Sure, Vital]",1.061,0.526316
597,4,"[Happy, Alert, Sure, Vital]",1.229,0.473684
598,4,"[Happy, Alert, Sure, Vital]",0.785,0.526316


In [71]:
candidate

{1: [['Alert'], ['Sure']],
 2: [['Alert', 'Sure'], ['Alert', 'Vital']],
 3: [['Alert', 'Sure', 'Vital']],
 4: [['Happy', 'Alert', 'Sure', 'Vital']]}

In [74]:
df_A = df_res.iloc[:100]
df_S = df_res.iloc[100:200]
df_AS = df_res.iloc[200:300]
df_AV = df_res.iloc[300:400]
df_ASV = df_res.iloc[400:500]
df_HASV = df_res.iloc[500:]

In [87]:
print('Alert:')
df_A.describe()

Alert:


,feature_num,MAPE,Direction
count,100.0,100.000000,100.000000
mean,1.0,1.703670,0.632941
std,0.0,1.048326,0.045886
min,1.0,0.664000,0.470588
25%,1.0,0.933750,0.588235
50%,1.0,1.483000,0.647059
75%,1.0,2.258000,0.647059
max,1.0,8.527000,0.764706


In [88]:
print('Sure: ')
df_S.describe()

Sure: 


,feature_num,MAPE,Direction
count,100.0,100.000000,100.000000
mean,1.0,2.697540,0.507619
std,0.0,1.179322,0.057518
min,1.0,0.734000,0.428571
25%,1.0,1.776750,0.476190
50%,1.0,2.953000,0.523810
75%,1.0,3.352250,0.523810
max,1.0,8.006000,0.761905


In [90]:
print('Alert, Sure: ')
df_AS.describe()

Alert, Sure: 


,feature_num,MAPE,Direction
count,100.0,100.000000,100.000000
mean,2.0,1.749560,0.601111
std,0.0,1.361606,0.062366
min,2.0,0.633000,0.500000
25%,2.0,0.895000,0.555556
50%,2.0,1.506000,0.611111
75%,2.0,2.053750,0.625000
max,2.0,8.374000,0.777778


In [91]:
print('Alert, Vital: ')
df_AV.describe()

Alert, Vital: 


,feature_num,MAPE,Direction
count,100.0,100.000000,100.000000
mean,2.0,1.586570,0.530556
std,0.0,1.073161,0.065294
min,2.0,0.690000,0.277778
25%,2.0,0.945250,0.500000
50%,2.0,1.214500,0.555556
75%,2.0,1.813500,0.555556
max,2.0,8.438000,0.777778


In [92]:
print('Sure, Alert, Vital: ' )
df_ASV.describe()

Sure, Alert, Vital: 


,feature_num,MAPE,Direction
count,100.0,100.00000,100.000000
mean,3.0,1.53256,0.560000
std,0.0,0.96315,0.058390
min,3.0,0.65400,0.388889
25%,3.0,0.93775,0.500000
50%,3.0,1.29800,0.555556
75%,3.0,1.82925,0.611111
max,3.0,8.38300,0.777778


In [93]:
print('Vital, Happy, Alert, Sure: ' )
df_HASV.describe()

Vital, Happy, Alert, Sure: 


,feature_num,MAPE,Direction
count,100.0,100.000000,100.000000
mean,4.0,1.518020,0.537895
std,0.0,0.810553,0.051056
min,4.0,0.738000,0.368421
25%,4.0,0.934750,0.526316
50%,4.0,1.209000,0.526316
75%,4.0,1.965250,0.578947
max,4.0,4.067000,0.684211


In [81]:
df_res

,feature_num,combination,MAPE,Direction
0,1,[Alert],2.347,0.647059
1,1,[Alert],0.820,0.470588
2,1,[Alert],2.095,0.647059
3,1,[Alert],1.961,0.647059
4,1,[Alert],1.099,0.647059
...,...,...,...,...
595,4,"[Happy, Alert, Sure, Vital]",2.596,0.473684
596,4,"[Happy, Alert, Sure, Vital]",1.061,0.526316
597,4,"[Happy, Alert, Sure, Vital]",1.229,0.473684
598,4,"[Happy, Alert, Sure, Vital]",0.785,0.526316


In [82]:
# Summary
combination = []
MAPE = []
Correction = []

j = 0
for i in range(100, 601, 100):
    tmp = df_res.iloc[j:i]
    tmp = tmp.groupby(by='feature_num').agg({'combination':'first',
                                              'MAPE':np.mean,
                                              'Direction':np.mean}).reset_index(drop=True)
    combination.append(tmp['combination'][0])
    MAPE.append(tmp['MAPE'][0])
    Correction.append(tmp['Direction'][0])
    j = i

df_final = pd.DataFrame({'combination':combination,
                         'MAPE':MAPE,
                         'Correction':Correction})

In [83]:
df_final

,combination,MAPE,Correction
0,[Alert],1.70367,0.632941
1,[Sure],2.69754,0.507619
2,"[Alert, Sure]",1.74956,0.601111
3,"[Alert, Vital]",1.58657,0.530556
4,"[Alert, Sure, Vital]",1.53256,0.560000
5,"[Happy, Alert, Sure, Vital]",1.51802,0.537895


reference: 

https://colab.research.google.com/drive/1b3CUJuDOmPmNdZFH3LQDmt5F0K3FZhqD?usp=sharing#scrollTo=FmwshpETs-jE